# Search Car Engine

In [ ]:
from sentence_transformers import SentenceTransformer
from constantS import TITLE_TO_VEC_FILE

import numpy as np

In [ ]:
class FindCar():
    
    def __init__(self):
        tmp_item = np.load(TITLE_TO_VEC_FILE, allow_pickle=True).item()
        self.title_embd_np_array = tmp_item["title_embd_array"]
        self.title_embd_norm_array = np.linalg.norm(self.title_embd_np_array, axis = 1)
        self.index_to_title_dict = tmp_item["index_to_title_dict"]
        
        self.encoder = SentenceTransformer('paraphrase-MiniLM-L6-v2')
        
    def recommend(self, query, top_k=5):
        query_emb = self.encoder.encode([query]).squeeze()
        output = ((self.title_embd_np_array @ query_emb)/self.title_embd_norm_array)/np.linalg.norm(query_emb)
        output_1 = output.argsort()[:top_k]
        
        return [self.index_to_title_dict[index] for index in output_1]

In [ ]:
Obj = FindCar()

In [ ]:
title_list = Obj.recommend(query)
for title in title_list:
    print(title)